In [20]:
library(tidyverse)
library(readxl)

# Read the Excel file
data <- read_excel("./data/exercice1.xlsx")

# Display the first few rows of the data
head(data)

ID,sexe,Region,Age,etat_civil,N_educatif,Membre_famille,Revenu_famille,Revenu_Mensuelle,Redoublant,⋯,Niveau_educ_pere,Niveau_educ_mere,Sign_ecole,Endroit_freq,Ph_famille,Ricule_Prof,Ricule_collegue,Sensation_apres_ecole,Apres_repture_ecole,Violence
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,1,2,1,3,5,0,0,1,⋯,2,2,1,1,1,0,1,1,1,1
2,0,1,4,2,3,5,0,0,1,⋯,2,1,1,2,2,1,1,1,1,1
3,0,1,4,2,3,11,0,100,1,⋯,2,1,2,1,3,0,0,1,2,0
5,0,1,3,2,3,4,0,0,1,⋯,1,1,1,1,2,1,1,3,3,1
8,0,1,1,1,3,5,1,0,1,⋯,3,3,3,4,4,0,0,3,2,1
9,0,1,4,3,3,4,1,100,0,⋯,2,3,4,1,4,1,1,1,2,0
